Create 4D heatmaps and store from 360 ERP + 7 audio heatmaps

In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import cv2
import soundfile as sf
import librosa
from scipy.integrate import dblquad
import os
from pathlib import Path

def compute_audio_at_direction(W: np.ndarray, X: np.ndarray,
                               Y: np.ndarray, Z: np.ndarray,
                               top_left: tuple, bottom_right: tuple,
                               center_time: float, sampleRate: int,
                               window_sec: float = 0.1) -> np.ndarray:
    """
    Compute audio waveform for a tile and around a specific time.

    Parameters:
        center_time: time in seconds to center the window
        window_sec: half-length of window (seconds) to extract
    """
    top_left_lat, top_left_lon = top_left
    bottom_right_lat, bottom_right_lon = bottom_right

    # Convert bounds to radians
    lat_min_rad = np.radians(bottom_right_lat)
    lat_max_rad = np.radians(top_left_lat)
    lon_min_rad = np.radians(top_left_lon)
    lon_max_rad = np.radians(bottom_right_lon)

    # Compute patch area (steradians)
    area = (lon_max_rad - lon_min_rad) * (np.sin(lat_max_rad) - np.sin(lat_min_rad))

    # Integrate Y_W over the region
    integral_Y_W, _ = dblquad(lambda lon, lat: np.cos(lat), lat_min_rad, lat_max_rad,
                              lambda _: lon_min_rad, lambda _: lon_max_rad)
    integral_Y_X, _ = dblquad(lambda lon, lat: np.cos(lat)*np.cos(lon)*np.cos(lat), lat_min_rad, lat_max_rad,
                              lambda _: lon_min_rad, lambda _: lon_max_rad)
    integral_Y_Y, _ = dblquad(lambda lon, lat: np.cos(lat)*np.sin(lon)*np.cos(lat), lat_min_rad, lat_max_rad,
                              lambda _: lon_min_rad, lambda _: lon_max_rad)
    integral_Y_Z, _ = dblquad(lambda lon, lat: np.sin(lat)*np.cos(lat), lat_min_rad, lat_max_rad,
                              lambda _: lon_min_rad, lambda _: lon_max_rad)

    # Convert time to sample indices
    center_sample = int(center_time * sampleRate)
    half_window_samples = int(window_sec * sampleRate)
    start = max(center_sample - half_window_samples, 0)
    end = min(center_sample + half_window_samples, len(W))

    # Extract the waveform slice
    W_slice = W[start:end]
    X_slice = X[start:end]
    Y_slice = Y[start:end]
    Z_slice = Z[start:end]

    # Reconstruct waveform for this tile
    wave = (integral_Y_W * W_slice + integral_Y_X * X_slice +
            integral_Y_Y * Y_slice + integral_Y_Z * Z_slice) / area

    return wave

def computeHNR(frame):
    """
    Compute HNR for a single frame using autocorrelation.
    HNR = 10 * log10(energy_harmonic / energy_noise)
    """
    frame = frame - np.mean(frame)  # remove DC
    if np.all(frame == 0):
        return 0.0

    # FFT-based autocorrelation
    autocorr = np.fft.irfft(np.fft.rfft(frame) * np.conj(np.fft.rfft(frame)))
    autocorr = autocorr / np.max(np.abs(autocorr))  # normalize

    # Harmonic energy = max autocorr (excluding lag 0)
    harmonic_energy = np.max(autocorr[1:])
    # Noise energy = lag 0 minus harmonic energy
    noise_energy = autocorr[0] - harmonic_energy
    if noise_energy <= 0:
        return 40.0  # cap to a reasonable max
    return 10 * np.log10(harmonic_energy / noise_energy)

def processWave(wave, sampleRate):
    windowSize = 2048
    hopSize = 100

    # converts this to a Short-Time Fourier Transform. Tells you how much eergy has at each frequency over time.
    # does this by going through windows. Length of each window defined by n_fft. Then, shifts window to right by length
    # hop length. at each window, computes how much of each frequency is present.
    # final value is 2D array of rows being each frequency, columns being time (which is now the windows), so value being amplitude/energy for that time and frequency
    stftWave = np.abs(librosa.stft(wave, n_fft=windowSize, hop_length=hopSize))
    # when we get the mel, that just converts all the frequencies to 128 possible onces, which are moreso frequencies humans can hear. So compressing
    # the frequencies from a large number of frequencies to a smaller number, in this case n_mels amount
    mel = librosa.feature.melspectrogram(S = stftWave, sr= sampleRate, n_mels = 128)
    # converts from power scaling of audio to decibel scaling, cause humans perceive in moreso logarithm of audio (so higher sounds kinda taper off to us)
    logMel = librosa.power_to_db(mel, ref=np.max)

    # gets overall frame energy, including amplitude
    volumeNorm = np.mean((logMel + 80), axis=0)
    # gets the contrast in energy between frequencies within a specific frequency band, so where some frequencies bands may have parts of high energy frequencies, while other parts are low energy
    contrast = librosa.feature.spectral_contrast(S = stftWave, sr=sampleRate)
    # combines the difference frequency bands to get a average contrast for that time frame
    contrast = np.mean(contrast, axis=0)
    # basically gets how much the sound chagnes over time. Does this by getting differnece over time fimes with np.diff, squaring that value, and getting its sum
    temporal_novelty = np.sum(np.diff(logMel, axis=1) ** 2, axis=0)
    # do this to add an extra value cause rn, the length is T - 1, since you're getting difference between frames. So add 1 to get it to T length
    temporal_novelty = np.insert(temporal_novelty, 0, 0)

    # this gets how noise like a sound is, whether it's tonal or liek white noise. A tonal sound is one that just
    # stands out, like through sharp peaks.
    spectral_flatness = librosa.feature.spectral_flatness(y=wave, n_fft=windowSize, hop_length= hopSize)
    # gets the average frequency weighted by amplitude, how "bright" the sound is, sees if the audio tends to have more high frequency or low frequency sounds
    centroid = librosa.feature.spectral_centroid(S=stftWave, sr=sampleRate)
    # indicates the range of frequencies present, so if the frequencies are more concentrated or spread out
    bandwidth = librosa.feature.spectral_bandwidth(S=stftWave, sr=sampleRate)



    # Compute HNR per frame. HNR is how harmonic the sound is, if its harmonic, with a pattern, or more noisy.
    # different from spectral flatness in that it measures if its harmonic, as opposed to tonal. Basically if there's like
    # a repeating pattern that identifies the town
    hnr_values = []
    num_frames = stftWave.shape[1]
    for i in range(num_frames):
        frame = wave[i*hopSize : i*hopSize + windowSize]
        if len(frame) < 2:
            continue
        hnr_values.append(computeHNR(frame))
    hnr_values = np.array(hnr_values)

    return np.array([np.mean(volumeNorm), np.mean(contrast), np.mean(temporal_novelty), np.mean(hnr_values), np.mean(spectral_flatness), np.mean(centroid), np.mean(bandwidth)])

def precompute_integrals(tile_size_deg=20):
    """
    Precompute integrals for coarse tiles (done once, cached for all frames).

    Parameters:
        tile_size_deg: size of each tile in degrees (default 20x20)

    Returns:
        Dictionary mapping (lat, lon) tile coordinates to integral values and area
    """
    print(f"Precomputing integrals for {tile_size_deg}° tiles...")

    tile_cache = {}

    # Generate all tile coordinates
    latitudes = list(range(90, -90, -tile_size_deg))
    longitudes = list(range(-180, 180, tile_size_deg))

    for topLeftLat in latitudes:
        for topLeftLon in longitudes:
            bottom_right_lat, bottom_right_lon = (topLeftLat - tile_size_deg, topLeftLon + tile_size_deg)

            # Convert bounds to radians
            lat_min_rad = np.radians(bottom_right_lat)
            lat_max_rad = np.radians(topLeftLat)
            lon_min_rad = np.radians(topLeftLon)
            lon_max_rad = np.radians(bottom_right_lon)

            # Compute patch area
            area = (lon_max_rad - lon_min_rad) * (np.sin(lat_max_rad) - np.sin(lat_min_rad))

            # Compute integrals
            integral_Y_W, _ = dblquad(
                lambda lon, lat: np.cos(lat), lat_min_rad, lat_max_rad,
                lambda _: lon_min_rad, lambda _: lon_max_rad)
            integral_Y_X, _ = dblquad(
                lambda lon, lat: np.cos(lat)*np.cos(lon)*np.cos(lat), lat_min_rad, lat_max_rad,
                lambda _: lon_min_rad, lambda _: lon_max_rad)
            integral_Y_Y, _ = dblquad(
                lambda lon, lat: np.cos(lat)*np.sin(lon)*np.cos(lat), lat_min_rad, lat_max_rad,
                lambda _: lon_min_rad, lambda _: lon_max_rad)
            integral_Y_Z, _ = dblquad(
                lambda lon, lat: np.sin(lat)*np.cos(lat), lat_min_rad, lat_max_rad,
                lambda _: lon_min_rad, lambda _: lon_max_rad)

            tile_cache[(topLeftLat, topLeftLon)] = (
                area,
                integral_Y_W,
                integral_Y_X,
                integral_Y_Y,
                integral_Y_Z
            )

    print("Integral precomputation complete!")
    return tile_cache


def get_tile_for_pixel(lat_pixel, lon_pixel, erp_height, erp_width, tile_size_deg=20):
    """
    Get the tile coordinates for a given pixel in the ERP map.
    """
    # Convert pixel to lat/lon
    lat = 90 - (lat_pixel / erp_height) * 180
    lon = -180 + (lon_pixel / erp_width) * 360

    # Find which tile this belongs to
    tile_lat = int(np.floor(lat / tile_size_deg)) * tile_size_deg
    tile_lon = int(np.floor(lon / tile_size_deg)) * tile_size_deg

    return (tile_lat, tile_lon)


def compute_audio_saliency_heatmap_vectorized(W, X, Y, Z, audio_samplerate,
                                              frame_idx, video_fps,
                                              erp_height, erp_width,
                                              tile_cache,  sample_every_n_frames,
                                              numHeatmaps, tile_size_deg=20):
    """
    Compute audio saliency heatmap for a given frame using precomputed tile integrals.
    Each tile's saliency is computed once and replicated to all pixels in that tile.
    Extracts audio from 2.5 frames before to 2.5 frames after the current frame.
    Returns a 2D array of shape (erp_height, erp_width).
    """
    # Time in seconds corresponding to this video frame
    time_sec = frame_idx / video_fps

    # Window: 2.5 frames before and 2.5 frames after = 5 frames total
    frameWindow = sample_every_n_frames / 2
    window_sec = frameWindow / video_fps

    # Convert time to sample indices
    center_sample = int(time_sec * audio_samplerate)
    half_window_samples = int(window_sec * audio_samplerate)
    start = max(center_sample - half_window_samples, 0)
    end = min(center_sample + half_window_samples, len(W))

    # Extract waveform slices
    W_slice = W[start:end]
    X_slice = X[start:end]
    Y_slice = Y[start:end]
    Z_slice = Z[start:end]

    # Initialize output saliency map
    saliency_map = np.zeros((numHeatmaps, erp_height, erp_width))

    numLatTiles = 180 // tile_size_deg
    numLonTiles = 360 // tile_size_deg

    # Calculate pixels per tile
    pixels_per_tile_lat = erp_height // numLatTiles  # 9 tiles in latitude
    pixels_per_tile_lon = erp_width // numLonTiles   # 18 tiles in longitude

    # Iterate through each tile
    for lat_tile in range(numLatTiles):
        for lon_tile in range(numLonTiles):
            # Get tile coordinates
            tile_lat = 90 - lat_tile * tile_size_deg
            tile_lon = -180 + lon_tile * tile_size_deg
            tile_coords = (tile_lat, tile_lon)

            if tile_coords in tile_cache:
                area, integral_Y_W, integral_Y_X, integral_Y_Y, integral_Y_Z = tile_cache[tile_coords]

                # Reconstruct waveform for this tile
                wave = (integral_Y_W * W_slice +
                       integral_Y_X * X_slice +
                       integral_Y_Y * Y_slice +
                       integral_Y_Z * Z_slice) / area

                if len(wave) > 0:
                    saliency_values = processWave(wave, audio_samplerate)
                else:
                    saliency_values = np.zeros(numHeatmaps)

                # Fill all pixels in this tile with the same saliency value
                y_start = lat_tile * pixels_per_tile_lat
                y_end = (lat_tile + 1) * pixels_per_tile_lat
                x_start = lon_tile * pixels_per_tile_lon
                x_end = (lon_tile + 1) * pixels_per_tile_lon

                # weirdly works cause of python's funky mapping
                saliency_map[:, y_start:y_end, x_start:x_end] = saliency_values[:, np.newaxis, np.newaxis]
            else:
                raise IndexError("IDk how we got this.")

    return saliency_map


def compute_video_saliency_heatmap_vectorized(frame_idx, video_fps,
                                              erp_height, erp_width,
                                              tile_cache,  sample_every_n_frames,
                                              numHeatmaps, tile_size_deg=20):
    """
    Compute video saliency heatmap for a given frame using ERP tile.
    Saliency is computed once on full ERP per frame.
    For motion/optical flow saliency use the current and previous frame (if exists)
    Returns a 2D array of shape (erp_height, erp_width).
    """
    # Time in seconds corresponding to this video frame
    time_sec = frame_idx / video_fps

    # Initialize output saliency map
    saliency_map = np.zeros((numHeatmaps, erp_height, erp_width))

    numLatTiles = 180 // tile_size_deg
    numLonTiles = 360 // tile_size_deg

    # Calculate pixels per tile
    pixels_per_tile_lat = erp_height // numLatTiles  # 9 tiles in latitude
    pixels_per_tile_lon = erp_width // numLonTiles   # 18 tiles in longitude

    # Example usage SalNet:
    # SalientStaticImage = process_image(Whereever image variable is)                    --- Image -> ERP Image -> ERP HeatMap
    # SalientStaticImageToTiles = saliency_to_rectified_tiles(saliency_full_resized)     --- ERP HeatMap -> ERP HeatMap Tiles

    # Example usage FlowNet:
    # img1 = cv2.cvtColor(cv2.imread("/content/frame1.jpg"), cv2.COLOR_BGR2RGB)          --- Frame 1
    # img2 = cv2.cvtColor(cv2.imread("/content/frame2.jpg"), cv2.COLOR_BGR2RGB)          --- Frame 2
    # flow = compute_flow(img1, img2, model)                                             --- Flow between Frame 1 & 2
    # heatmap_img = generate_flow_heatmap(flow)                                          --- HeatMap from Flow

    return saliency_map


def normalize_heatmaps(heatmaps):
    """Normalize heatmap to [0, 1] range."""
    # returns a list of mins and maxs for each heatmap
    h_mins = np.min(heatmaps, axis=(1, 2), keepdims=True)
    h_maxs = np.max(heatmaps, axis=(1, 2), keepdims=True)

    return (heatmaps - h_mins) / (h_maxs - h_mins)


def getFrame(cap, output_width, output_height, frame_idx):
    """
    Read video and yield resized frames.
    """

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    resized_frame = cv2.resize(frame, (output_width, output_height), interpolation=cv2.INTER_LINEAR)

    return resized_frame

def process_360_video(video_path, audio_path, output_path,
                      erp_height=1920, erp_width=3840, sample_every_n_frames=5, numHeatmaps=9):
    """
    Main pipeline to process a 360 video and extract audio saliency heatmaps.

    Parameters:
        video_path: path to ERP format 360 video
        audio_path: path to first-order ambisonic audio file
        output_path: where to save the output .npy file
        erp_height: height of ERP format (pixels)
        erp_width: width of ERP format (pixels)
        sample_every_n_frames: sample every N frames
    """

    # Load audio
    print("Loading ambisonic audio...")
    audio_data, audio_samplerate = sf.read(audio_path)

    # Check for 4 channels
    if len(audio_data.shape) == 1:
        raise ValueError(f"Audio is mono. Expected 4-channel first-order ambisonics.")
    elif audio_data.shape[1] != 4:
        raise ValueError(f"Audio has {audio_data.shape[1]} channels. Expected 4-channel first-order ambisonics (W, X, Y, Z).")

    # Split into channels
    W = audio_data[:, 0]
    X = audio_data[:, 1]
    Y = audio_data[:, 2]
    Z = audio_data[:, 3]

    print(f"Audio shape: {audio_data.shape}")
    print(f"Audio sample rate: {audio_samplerate} Hz")
    print("Successfully loaded 4-channel first-order ambisonics audio")

    # Open video to get metadata
    print("Opening video...")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Video FPS: {video_fps}")
    print(f"Total frames: {total_frames}")
    print(f"Video dimensions: {video_width}x{video_height}")
    print(f"Target ERP dimensions: {erp_width}x{erp_height}")

    # Check if resizing is needed
    need_resize = video_width != erp_width or video_height != erp_height
    if need_resize:
        print(f"Video will be resized from {video_width}x{video_height} to {erp_width}x{erp_height}")

    # Precompute integrals for coarse tiles (20x20 degrees)
    tile_cache = precompute_integrals(tile_size_deg=20)

    # Calculate number of sampled frames
    num_sampled_frames = (total_frames - math.ceil(sample_every_n_frames / 2)) // sample_every_n_frames

    # Initialize output array
    output_array = np.zeros((num_sampled_frames, numHeatmaps, erp_height, erp_width), dtype=np.float16)

    print(f"Output array shape: {output_array.shape}")
    print(f"Processing {num_sampled_frames} frames...")

    # Use frame generator (resizes all frames upfront in the stream). Also, only retrieves them one at a time, instead of keeping it all in memory
    for sampled_frame_idx in range(num_sampled_frames):
        frame_idx = sample_every_n_frames * (sampled_frame_idx + 1)

        frame = getFrame(cap, erp_height, erp_width, frame_idx)

        print(f"Processing frame {frame_idx}/{total_frames} (sample {sampled_frame_idx}/{num_sampled_frames})")


        # Compute audio-visual saliency heatmap
        saliency_heatmaps = np.concat(compute_audio_saliency_heatmap_vectorized(W, X, Y, Z, audio_samplerate,
                                                                        frame_idx, video_fps,
                                                                        erp_height, erp_width,
                                                                        tile_cache, sample_every_n_frames,
                                                                        numHeatmaps-2, tile_size_deg=20),
                                      compute_video_saliency_heatmap_vectorized(frame_idx, video_fps,
                                                                        erp_height, erp_width,
                                                                        tile_cache, sample_every_n_frames,
                                                                        numHeatmaps-7, tile_size_deg=20)
                                      )

        # Normalize heatmap
        saliency_heatmaps = normalize_heatmaps(saliency_heatmaps)

        # Store in output array
        output_array[sampled_frame_idx] = saliency_heatmaps

    cap.release()

    # Save output
    print(f"Saving output to {output_path}...")
    np.save(output_path, output_array)

    print(f"Done! Output shape: {output_array.shape}")
    print(f"Saved to: {output_path}")

    return output_array

if __name__ == "__main__":
    os.chdir("./../..")

    # Configuration - modify as needed
    ERP_WIDTH = 3840  # width
    ERP_HEIGHT = 1920  # height
    SAMPLE_RATE = 5  # sample every 5 frames
    FILE_NAME = "5020"
    VIDEO_PATH = f"Data/Pre-Processed-Data/{FILE_NAME}.mp4"  # ERP format 360 video
    AUDIO_PATH = f"Data/Pre-Processed-Data/{FILE_NAME}.wav"
    OUTPUT_PATH = f"FinalTrainingData/{FILE_NAME}.npy"
    numHeatmaps = 9

    # Run the pipeline
    saliency_array = process_360_video(VIDEO_PATH, AUDIO_PATH, OUTPUT_PATH,
                                      erp_height=1920, erp_width=3840,
                                      sample_every_n_frames=5, numHeatmaps=9)

Loading ambisonic audio...


LibsndfileError: Error opening 'Data/Pre-Processed-Data/5020.wav': System error.

Video Static + Motion/Optical Flow Saliency

In [ ]:
from IPython.display import Image as ColabImage
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import torch
import torch.nn.functional as F
from torchvision import transforms
import os

# ============================================================
# STEP 0: Setup
# ============================================================

MODEL_PATH = "/content/U-2-Net/u2netp.pth"
URL_DL = "https://drive.google.com/uc?export=download&id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy"

# Ensure folder exists
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

# Download if missing
if not os.path.exists(MODEL_PATH):
    print("⬇ Downloading U2NetP weights…")
    result = os.system(f"wget -q --no-check-certificate '{URL_DL}' -O {MODEL_PATH}")
    if result != 0 or not os.path.exists(MODEL_PATH):
        raise FileNotFoundError("❌ Failed to download U2NetP.")
    print("✔ Download succeeded! Saved to:", MODEL_PATH)
else:
    print("✔ Using cached u2netp.pth")


!git clone https://github.com/xuebinqin/U-2-Net.git /content/U-2-Net-Repo
!pip install -q torch torchvision
import sys
sys.path.append('/content/U-2-Net-Repo')
from model.u2net import U2NETP


# Then load model as before
u2netp = U2NETP(3, 1)
u2netp.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
u2netp.eval()
print("✔ U2 Net Model ready!")

transform_u2 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((320, 320)),
    transforms.ToTensor()
])

# ============================================================
# STEP 1–4: Define Processing Function
# ============================================================

def process_image(filename):
    print("\n" + "="*60)
    print(f"Processing: {filename}")
    print("="*60)

    # STEP 1: Load ERP Image
    erp = cv2.imread(filename)
    if erp is None:
        print(f"⚠️ File not found: {filename}")
        return
    erp = cv2.cvtColor(erp, cv2.COLOR_BGR2RGB)
    H, W, _ = erp.shape
    print(f"Height = {H}, Width = {W}")

    plt.figure(figsize=(10, 5))
    plt.imshow(erp)
    plt.title(f"Original ERP: {os.path.basename(filename)}")
    plt.axis("off")
    plt.show()

    # STEP 2: Rectified Tiling (20°×20°)
    widthDegree, heightDegree = 20, 20
    num_lon_tiles = int(360 / widthDegree)
    num_lat_tiles = int(180 / heightDegree)

    tileWidth = int(round(W / num_lon_tiles))
    tileHeight = int(round(H / num_lat_tiles))
    print(f"Tile width = {tileWidth}, Tile height = {tileHeight}")

    finalTiles = [[None for _ in range(num_lon_tiles)] for _ in range(num_lat_tiles)]

    for lat_i in range(num_lat_tiles):
        lat_max = 90.0 - lat_i * heightDegree
        lat_min = lat_max - heightDegree
        lat_vals = np.linspace(lat_max, lat_min, tileHeight)

        for lon_j in range(num_lon_tiles):
            lon_min = lon_j * widthDegree - 180.0
            lon_max = lon_min + widthDegree
            lon_vals = np.linspace(lon_min, lon_max, tileWidth)

            erp_x_vec = (lon_vals + 180.0) / 360.0 * (W - 1)
            erp_y_vec = (90.0 - lat_vals) / 180.0 * (H - 1)

            map_x = np.tile(erp_x_vec, (tileHeight, 1)).astype(np.float32)
            map_y = np.repeat(erp_y_vec[:, np.newaxis], tileWidth, axis=1).astype(np.float32)

            tile_img = cv2.remap(erp, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
            finalTiles[lat_i][lon_j] = tile_img

    # STEP 3: Reconstruct ERP from Rectified Tiles
    erp_reconstructed = np.vstack([np.hstack(row) for row in finalTiles])
    #print("Reconstructed ERP shape:", erp_reconstructed.shape)

    #plt.figure(figsize=(12, 6))
    #plt.imshow(erp_reconstructed)
    #plt.title("Reconstructed ERP from 20°×20° Rectified Tiles")
    #plt.axis("off")
    #plt.show()

    # STEP 4: Run U²-NetP (Global)
    input_full = transform_u2(erp_reconstructed).unsqueeze(0)
    with torch.no_grad():
        d1, *_ = u2netp(input_full)
        pred_full = F.interpolate(d1, size=(H, W), mode="bilinear", align_corners=False)
        saliency_full = pred_full.squeeze().cpu().numpy()

    saliency_full_resized = (saliency_full - saliency_full.min()) / (saliency_full.max() - saliency_full.min() + 1e-8)

    #plt.figure(figsize=(12, 6))
    #plt.imshow(saliency_full_resized, cmap="inferno")
    #plt.title("U²-NetP Saliency Map (Full ERP)")
    #plt.axis("off")
    #plt.show()

    return saliency_full_resized

# Step 5: Turn Salient Image into Tiles

def saliency_to_rectified_tiles(saliency_map, widthDegree=20, heightDegree=20):
    H, W = saliency_map.shape
    num_lon_tiles = int(360 / widthDegree)
    num_lat_tiles = int(180 / heightDegree)

    tileWidth = int(round(W / num_lon_tiles))
    tileHeight = int(round(H / num_lat_tiles))

    tiles = [[None for _ in range(num_lon_tiles)] for _ in range(num_lat_tiles)]

    for lat_i in range(num_lat_tiles):
        lat_max = 90.0 - lat_i * heightDegree
        lat_min = lat_max - heightDegree
        lat_vals = np.linspace(lat_max, lat_min, tileHeight)

        for lon_j in range(num_lon_tiles):
            lon_min = lon_j * widthDegree - 180.0
            lon_max = lon_min + widthDegree
            lon_vals = np.linspace(lon_min, lon_max, tileWidth)

            erp_x_vec = (lon_vals + 180.0) / 360.0 * (W - 1)
            erp_y_vec = (90.0 - lat_vals) / 180.0 * (H - 1)

            map_x = np.tile(erp_x_vec, (tileHeight, 1)).astype(np.float32)
            map_y = np.repeat(erp_y_vec[:, np.newaxis], tileWidth, axis=1).astype(np.float32)

            tile_sal = cv2.remap(
                saliency_map.astype(np.float32),
                map_x,
                map_y,
                interpolation=cv2.INTER_LINEAR,
                borderMode=cv2.BORDER_WRAP
            )

            tiles[lat_i][lon_j] = tile_sal

    return tiles


# ============================================================
# RAFT SETUP (Using HaniKamran/RAFT Fork)
# ============================================================

import os, sys
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt
import argparse

# ---------------------------
# Setup working directory and Clone your Patched Repo
# ---------------------------
os.chdir("/content")

# Install minimal dependencies
print("Installing dependencies...")
# Ensure scipy is installed correctly to avoid earlier conflicts
!pip install -q opencv-python matplotlib Pillow einops scipy

# Remove old RAFT if exists and clone YOUR FORK
!rm -rf /content/RAFT
# Cloning your specific patched branch: HaniKamran-patch-1
!git clone https://github.com/HaniKamran/RAFT.git /content/RAFT
%cd /content/RAFT
!git checkout HaniKamran-patch-1
%cd /content

# Add RAFT core modules to path
sys.path.append("/content/RAFT/core")

# ---------------------------
# Import Patched RAFT Modules
# ---------------------------
try:
    from raft import RAFT
    from utils.utils import InputPadder
    print("✓ Successfully imported core RAFT modules from your fork.")
except ImportError as e:
    print(f"FATAL ERROR: Could not import RAFT modules. Check your fork structure: {e}")
    sys.exit(1)


# ---------------------------
# Load CPU-only RAFT model
# ---------------------------
# These arguments trigger the 'small' model and disable CUDA-specific features.
# The custom logic in your raft.py will ensure the CPUCostVolume is used here.
args = argparse.Namespace(
    small=True,
    mixed_precision=False,
    alternate_corr=False, # Set to False to use the primary CorrBlock path (which you patched)
    dropout=0,
    dropout2=0,
)

weights_path = "/content/RAFT/models/raft-small.pth"

# Download weights if missing
if not os.path.exists(weights_path):
    print("Downloading RAFT weights...")
    %cd /content/RAFT
    !bash download_models.sh
    %cd /content

print("Loading RAFT CPU model...")
# Initialize model on CPU
model = RAFT(args).cpu()

# Clean state_dict and load weights
checkpoint = torch.load(weights_path, map_location="cpu")
state_dict = checkpoint.get("state_dict", checkpoint)
clean_state = {}
for k, v in state_dict.items():
    new_k = k.replace("module.", "")
    if new_k in model.state_dict():
        clean_state[new_k] = v

model.load_state_dict(clean_state, strict=False)
model.eval()
print("✓ RAFT model loaded on CPU successfully.")

# ---------------------------
# CPU-safe optical flow function
# ---------------------------
def compute_flow_debug(img1, img2, model):
    """Computes optical flow using the loaded RAFT model across various scales."""
    import matplotlib.pyplot as plt

    original_h, original_w = img1.shape[:2]
    # Use smaller scales for better CPU stability and speed
    scales = [1.0, 0.75, 0.5, 0.33, 0.25, 0.1, .01]

    for s in scales:
        try:
            print(f"\nTrying scale {s}...")
            # Ensure dimensions are at least 32x32 and multiples of 8 (padding will handle multiples of 8)
            new_h, new_w = max(32, int(original_h * s)), max(32, int(original_w * s))
            im1 = cv2.resize(img1, (new_w, new_h))
            im2 = cv2.resize(img2, (new_w, new_h))

            # Convert to tensor and normalize [0, 1]
            t1 = torch.from_numpy(im1/255.).permute(2,0,1).float().unsqueeze(0)
            t2 = torch.from_numpy(im2/255.).permute(2,0,1).float().unsqueeze(0)

            # Pad to multiples of 8 (required by RAFT architecture)
            padder = InputPadder(t1.shape)
            t1, t2 = padder.pad(t1, t2)

            print(f"Padded tensor shapes: t1={t1.shape}, t2={t2.shape}")

            # Run RAFT
            with torch.no_grad():
                # Setting iters=12 is faster for inference than the default 32
                _, flow_up = model(t1, t2, iters=12, test_mode=True)

            # Post-process flow: Resize back to original resolution (H, W)
            flow = flow_up[0].permute(1,2,0).cpu().numpy()

            # The flow values need to be scaled correctly after resizing
            flow_x = cv2.resize(flow[...,0], (original_w, original_h), interpolation=cv2.INTER_LINEAR) * (original_w / flow.shape[1])
            flow_y = cv2.resize(flow[...,1], (original_w, original_h), interpolation=cv2.INTER_LINEAR) * (original_h / flow.shape[0])
            flow_final = np.stack([flow_x, flow_y], axis=-1)

            print(f"✓ Flow computed successfully at scale {s}")
            return flow_final

        except Exception as e:
            print(f"Failed at scale {s} with error: {e}")

    raise RuntimeError("Flow failed at all scales")

print("\n✓ RAFT CPU flow ready for execution. You can now call the function.")

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def flow_to_magnitude_map(flow):
    """
    Computes only the magnitude (speed) of the optical flow.
    Returns a normalized single-channel array.
    """
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    rad = np.sqrt(u**2 + v**2)
    flow_max = np.max(rad)
    if flow_max > 0:
        rad /= flow_max

    return rad.astype(np.float32)


def generate_flow_heatmap(flow):
    """
    Returns the heatmap image (RGB uint8 array) for the flow magnitude.
    Does NOT display the heatmap.
    """
    magnitude_map = flow_to_magnitude_map(flow)

    # Apply 'jet' colormap using matplotlib
    colormap = cm.get_cmap('jet')
    heatmap = colormap(magnitude_map)  # RGBA float32 in [0,1]

    # Convert RGBA float [0,1] → RGB uint8 [0,255]
    heatmap = (heatmap[:, :, :3] * 255).astype(np.uint8)

    return heatmap

#heatmap_img = generate_flow_heatmap(flow)

#plt.imshow(heatmap_img)
#plt.title("Flow Heatmap (Returned Variable)")
#plt.axis("off")
#plt.show()


# Example usage SalNet:
# SalientStaticImage = process_image(Whereever image variable is)                    --- Image -> ERP Image -> ERP HeatMap
# SalientStaticImageToTiles = saliency_to_rectified_tiles(saliency_full_resized)     --- ERP HeatMap -> ERP HeatMap Tiles

# Example usage FlowNet:
# img1 = cv2.cvtColor(cv2.imread("/content/frame1.jpg"), cv2.COLOR_BGR2RGB)          --- Frame 1
# img2 = cv2.cvtColor(cv2.imread("/content/frame2.jpg"), cv2.COLOR_BGR2RGB)          --- Frame 2
# flow = compute_flow(img1, img2, model)                                             --- Flow between Frame 1 & 2
# heatmap_img = generate_flow_heatmap(flow)                                          --- HeatMap from Flow

1D head position index data processing

In [ ]:
import pandas as pd
import numpy as np
import os

def process_head_tracking_data(input_csv_path, participant_id, video_name,
                                fps=60, frame_interval=5, cols=16, rows=9):
    """
    Process head tracking data and convert to tile indices

    Parameters:
    -----------
    input_csv_path : str
        Path to the input CSV file
    participant_id : int
        ID of the participant to filter
    video_name : int
        Video identifier to filter
    fps : int
        Frames per second (default: 60)
    frame_interval : int
        Take every nth frame (default: 5)
    cols : int
        Number of columns in tile grid (default: 16)
    rows : int
        Number of rows in tile grid (default: 9)

    Returns:
    --------
    tile_indices_array : numpy array
        1D array of tile indices
    target_timestamps : numpy array
        Corresponding timestamps for each tile index
    """

    # Read the CSV file
    print(f"Reading CSV file: {input_csv_path}")
    df = pd.read_csv(input_csv_path)

    # Filter for specific participant and video
    filtered_df = df[(df['id'] == participant_id) & (df['video'] == video_name)]

    if len(filtered_df) == 0:
        raise ValueError(f"No data found for participant {participant_id} and video {video_name}")

    # Check stereo mode
    stereo_mode = filtered_df['stereo'].iloc[0]
    mode_str = "stereo" if stereo_mode else "mono"

    # Sort by timestamp to ensure correct ordering
    filtered_df = filtered_df.sort_values('t').reset_index(drop=True)

    print(f"\nProcessing:")
    print(f"  Participant ID: {participant_id}")
    print(f"  Video: {video_name}")
    print(f"  Mode: {mode_str}")
    print(f"  Samples in filtered data: {len(filtered_df)}")

    # Parameters
    time_interval = frame_interval / fps  # Time between samples

    # Function to convert (u, v) coordinates to tile index
    def uv_to_tile_index(u, v):
        """Convert normalized coordinates (u, v) to tile index"""
        col = int(u * cols)
        row = int(v * rows)

        # Clamp to valid range
        col = min(col, cols - 1)
        row = min(row, rows - 1)

        # Calculate tile index (row-major order)
        tile_index = row * cols + col
        return tile_index

    # Get video duration (max timestamp)
    max_time = filtered_df['t'].max()

    # Generate target timestamps (every nth frame)
    target_timestamps = np.arange(0, max_time + time_interval, time_interval)

    print(f"  Video duration: {max_time:.2f} seconds")
    print(f"  Target timestamps: {len(target_timestamps)} (every {frame_interval} frames)")

    # Find closest timestamp in data for each target timestamp
    tile_indices = []

    for target_t in target_timestamps:
        # Find the row with closest timestamp
        idx = (filtered_df['t'] - target_t).abs().idxmin()

        # Get u, v coordinates
        u = filtered_df.loc[idx, 'u']
        v = filtered_df.loc[idx, 'v']

        # Convert to tile index
        tile_idx = uv_to_tile_index(u, v)
        tile_indices.append(tile_idx)

    # Convert to numpy array
    tile_indices_array = np.array(tile_indices)

    print(f"\nGenerated {len(tile_indices_array)} tile indices")
    print(f"Tile index range: {tile_indices_array.min()} to {tile_indices_array.max()}")

    return tile_indices_array, target_timestamps, mode_str


def save_tile_indices_to_csv(tile_indices, timestamps, participant_id, video_name,
                              mode_str, fps, frame_interval, output_dir='.'):
    """
    Save tile indices to CSV file with proper naming (includes metadata)

    Parameters:
    -----------
    tile_indices : numpy array
        1D array of tile indices
    timestamps : numpy array
        Corresponding timestamps
    participant_id : int
        ID of the participant
    video_name : int
        Video identifier
    mode_str : str
        'mono' or 'stereo'
    fps : int
        Frames per second
    frame_interval : int
        Frame interval used
    output_dir : str
        Output directory (default: current directory)
    """

    # Format video name with leading zeros (e.g., 1 -> 0001)
    video_str = f"{video_name:04d}"

    # Create filename
    filename = f"id{participant_id}_head_video_{video_str}_{mode_str}_{fps}fps_{frame_interval}th_frames.csv"
    output_path = os.path.join(output_dir, filename)

    # Create DataFrame with tile indices and timestamps
    output_df = pd.DataFrame({
        'frame_index': np.arange(0, len(tile_indices) * frame_interval, frame_interval),
        'timestamp': timestamps,
        'tile_index': tile_indices
    })

    # Save to CSV
    output_df.to_csv(output_path, index=False)
    print(f"\nOutput saved to: {output_path}")
    print(f"Output shape: {output_df.shape}")
    print(f"\nFirst 10 rows of output:")
    print(output_df.head(10))

    return output_path


def save_tile_indices_cnn_format(tile_indices, participant_id, video_name,
                                  mode_str, fps, frame_interval, output_dir='.'):
    """
    Save tile indices as a single line CSV for CNN input (no metadata, no headers)

    Parameters:
    -----------
    tile_indices : numpy array
        1D array of tile indices
    participant_id : int
        ID of the participant
    video_name : int
        Video identifier
    mode_str : str
        'mono' or 'stereo'
    fps : int
        Frames per second
    frame_interval : int
        Frame interval used
    output_dir : str
        Output directory (default: current directory)
    """

    # Format video name with leading zeros (e.g., 1 -> 0001)
    video_str = f"{video_name:04d}"

    # Create filename with _cnn suffix
    filename = f"id{participant_id}_head_video_{video_str}_{mode_str}_{fps}fps_{frame_interval}th_frames_cnn.csv"
    output_path = os.path.join(output_dir, filename)

    # Save as single line without header or index
    np.savetxt(output_path, tile_indices.reshape(1, -1), delimiter=',', fmt='%d')

    print(f"\nCNN format output saved to: {output_path}")
    print(f"Array length: {len(tile_indices)}")
    print(f"First 20 tile indices: {tile_indices[:20]}")

    return output_path


# ==================== MAIN EXECUTION ====================

# Configuration
INPUT_CSV_PATH = 'your_data.csv'  # Change this to your CSV file path
PARTICIPANT_ID = 11
VIDEO_NAME = 1  # Video 001
FPS = 60
FRAME_INTERVAL = 5
TILE_COLS = 16
TILE_ROWS = 9

# For Google Colab: Upload file
print("Upload your CSV file:")
from google.colab import files
uploaded = files.upload()
INPUT_CSV_PATH = list(uploaded.keys())[0]

# Process the data
tile_indices, timestamps, mode_str = process_head_tracking_data(
    input_csv_path=INPUT_CSV_PATH,
    participant_id=PARTICIPANT_ID,
    video_name=VIDEO_NAME,
    fps=FPS,
    frame_interval=FRAME_INTERVAL,
    cols=TILE_COLS,
    rows=TILE_ROWS
)

# Save to CSV with metadata (original format)
output_path = save_tile_indices_to_csv(
    tile_indices=tile_indices,
    timestamps=timestamps,
    participant_id=PARTICIPANT_ID,
    video_name=VIDEO_NAME,
    mode_str=mode_str,
    fps=FPS,
    frame_interval=FRAME_INTERVAL
)

# Save to CSV for CNN input (single line, no headers)
output_path_cnn = save_tile_indices_cnn_format(
    tile_indices=tile_indices,
    participant_id=PARTICIPANT_ID,
    video_name=VIDEO_NAME,
    mode_str=mode_str,
    fps=FPS,
    frame_interval=FRAME_INTERVAL
)

# Optional: Visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.plot(timestamps, tile_indices, marker='o', markersize=2, linestyle='-', linewidth=0.5)
plt.xlabel('Time (seconds)')
plt.ylabel('Tile Index')
plt.title(f'Tile Indices Over Time\n(ID={PARTICIPANT_ID}, Video={VIDEO_NAME:04d})')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.hist(tile_indices, bins=TILE_COLS * TILE_ROWS, range=(0, TILE_COLS * TILE_ROWS))
plt.xlabel('Tile Index')
plt.ylabel('Frequency')
plt.title('Distribution of Viewed Tiles')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
# Visualize as heatmap
tile_counts = np.bincount(tile_indices, minlength=TILE_COLS * TILE_ROWS)
heatmap = tile_counts.reshape(TILE_ROWS, TILE_COLS)
plt.imshow(heatmap, cmap='hot', interpolation='nearest', aspect='auto')
plt.colorbar(label='View Count')
plt.xlabel('Tile Column')
plt.ylabel('Tile Row')
plt.title('Heatmap of Viewed Tiles')
plt.xticks(range(0, TILE_COLS, 2))
plt.yticks(range(TILE_ROWS))

plt.tight_layout()
plt.show()

# Download both output files (for Colab)
# print("\n" + "="*60)
# print("Downloading output files...")
# print("="*60)
# files.download(output_path)
# files.download(output_path_cnn)
# ```

# **Now you get TWO output files:**

# 1. **`id11_head_video_0001_mono_60fps_5th_frames.csv`** (with metadata):
# ```
# frame_index,timestamp,tile_index
# 0,0.0,67
# 5,0.0833333,68
# 10,0.1666667,69
# ...
# ```

# 2. **`id11_head_video_0001_mono_60fps_5th_frames_cnn.csv`** (for CNN input):
# ```
# 67,68,69,70,71,68,67,65,64,63,62,61,60,59,58,57,56,55,54,53,...

CNN

In [ ]:
!pip install torch torchvision tqdm tabulate --quiet

import time, os, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from tabulate import tabulate
from tqdm import tqdm
import gc

# Clear GPU memory
torch.cuda.empty_cache() if torch.cuda.is_available() else None

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\n")

# Configuration - 25% of 4K resolution
FRAME_WIDTH = 960    # 25% of 3840
FRAME_HEIGHT = 480   # 25% of 1920
TILES_X = 16
TILES_Y = 9
NUM_TILES = TILES_X * TILES_Y  # 144 tiles
TILE_WIDTH = FRAME_WIDTH // TILES_X   # 60 pixels
TILE_HEIGHT = FRAME_HEIGHT // TILES_Y  # ~53 pixels
NUM_HEATMAPS = 9  # 7 audio + 2 video

# Video configuration
FPS = 60  # Frames per second (full framerate)
VIDEO_DURATION = 3  # seconds
FRAME_SAMPLE_RATE = 5  # Take every 5th frame
TOTAL_FRAMES = FPS * VIDEO_DURATION  # 180 frames total for 3 seconds
FRAMES_PER_VIDEO = TOTAL_FRAMES // FRAME_SAMPLE_RATE  # 36 sampled frames

class SaliencyTileDataset(Dataset):
    """Memory-efficient dataset for saliency heatmaps"""

    def __init__(self, heatmaps, tile_indices):
        """
        Args:
            heatmaps: numpy array of shape (N_frames, NUM_HEATMAPS, H, W)
            tile_indices: numpy array of shape (N_frames,) with tile indices [0-143]
        """
        # Convert to float32 for memory efficiency
        self.heatmaps = heatmaps.astype(np.float32)
        self.tile_indices = tile_indices.astype(np.int64)

    def __len__(self):
        return len(self.tile_indices)

    def __getitem__(self, idx):
        heatmap = torch.from_numpy(self.heatmaps[idx])
        tile_idx = torch.tensor(self.tile_indices[idx], dtype=torch.long)
        return heatmap, tile_idx


class HeatmapFusionCNN(nn.Module):
    """Lightweight CNN for fusing saliency heatmaps"""

    def __init__(self, num_heatmaps=9, num_tiles=144, dropout=0.3):
        super(HeatmapFusionCNN, self).__init__()

        # Heatmap fusion with 1x1 conv
        self.fusion = nn.Conv2d(num_heatmaps, 8, kernel_size=1)

        # Lightweight feature extraction
        self.conv1 = nn.Conv2d(8, 32, kernel_size=5, stride=4, padding=2)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # Adaptive pooling
        self.pool = nn.AdaptiveAvgPool2d((4, 4))

        # Classifier
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(256, num_tiles)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # Fuse heatmaps
        x = self.fusion(x)
        x = self.relu(x)

        # Feature extraction
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        # Pool and classify
        x = self.pool(x)
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x


def tile_coords_to_index(x, y, tiles_x=TILES_X):
    """Convert tile coordinates to linear index"""
    return y * tiles_x + x


def tile_index_to_coords(idx, tiles_x=TILES_X):
    """Convert linear index to tile coordinates"""
    y = idx // tiles_x
    x = idx % tiles_x
    return x, y


def tile_distance(pred_idx, true_idx, tiles_x=TILES_X):
    """Calculate tile distance"""
    px, py = tile_index_to_coords(pred_idx, tiles_x)
    tx, ty = tile_index_to_coords(true_idx, tiles_x)
    return np.sqrt((px - tx)**2 + (py - ty)**2)


def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for heatmaps, tile_indices in dataloader:
        heatmaps = heatmaps.to(device)
        tile_indices = tile_indices.to(device)

        optimizer.zero_grad()
        outputs = model(heatmaps)
        loss = criterion(outputs, tile_indices)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += tile_indices.size(0)
        correct += predicted.eq(tile_indices).sum().item()

    return total_loss / len(dataloader), 100. * correct / total


def validate(model, dataloader, criterion, device):
    """Validate the model"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    tile_distances = []

    with torch.no_grad():
        for heatmaps, tile_indices in dataloader:
            heatmaps = heatmaps.to(device)
            tile_indices = tile_indices.to(device)

            outputs = model(heatmaps)
            loss = criterion(outputs, tile_indices)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += tile_indices.size(0)
            correct += predicted.eq(tile_indices).sum().item()

            for pred, true in zip(predicted.cpu().numpy(), tile_indices.cpu().numpy()):
                tile_distances.append(tile_distance(pred, true))

    avg_distance = np.mean(tile_distances)
    return total_loss / len(dataloader), 100. * correct / total, avg_distance


def generate_synthetic_video_data(num_videos=1):
    """
    Generate synthetic video data at 60 FPS, sampling every 5th frame

    Returns:
        heatmaps_4d: shape (num_videos, FRAMES_PER_VIDEO, NUM_HEATMAPS, H, W)
        tile_indices_1d: shape (num_videos, FRAMES_PER_VIDEO)
    """
    print(f"\n{'='*60}")
    print(f"Generating Synthetic Video Data")
    print(f"{'='*60}\n")
    print(f"Videos: {num_videos}")
    print(f"Duration: {VIDEO_DURATION}s @ {FPS} FPS = {TOTAL_FRAMES} total frames")
    print(f"Sampling: Every {FRAME_SAMPLE_RATE}th frame = {FRAMES_PER_VIDEO} sampled frames")
    print(f"Heatmap resolution: {FRAME_WIDTH}x{FRAME_HEIGHT} (25% of 4K)")
    print(f"Total sampled frames: {num_videos * FRAMES_PER_VIDEO}\n")

    # Initialize 4D matrix for heatmaps: (videos, sampled_frames, heatmaps, height, width)
    heatmaps_4d = np.zeros((num_videos, FRAMES_PER_VIDEO, NUM_HEATMAPS,
                            FRAME_HEIGHT, FRAME_WIDTH), dtype=np.float32)

    # Initialize 1D matrix for tile indices: (videos, sampled_frames)
    tile_indices_1d = np.zeros((num_videos, FRAMES_PER_VIDEO), dtype=np.int64)

    for video_idx in range(num_videos):
        print(f"Generating video {video_idx + 1}/{num_videos}...")

        # Create a smooth trajectory for the salient region across ALL frames
        start_x = np.random.randint(TILE_WIDTH * 2, FRAME_WIDTH - TILE_WIDTH * 2)
        start_y = np.random.randint(TILE_HEIGHT * 2, FRAME_HEIGHT - TILE_HEIGHT * 2)

        end_x = np.random.randint(TILE_WIDTH * 2, FRAME_WIDTH - TILE_WIDTH * 2)
        end_y = np.random.randint(TILE_HEIGHT * 2, FRAME_HEIGHT - TILE_HEIGHT * 2)

        # Create smooth trajectory for ALL 180 frames (60 FPS * 3 sec)
        trajectory_x = np.linspace(start_x, end_x, TOTAL_FRAMES)
        trajectory_y = np.linspace(start_y, end_y, TOTAL_FRAMES)

        # Add jitter
        jitter_x = np.random.randn(TOTAL_FRAMES) * (TILE_WIDTH / 4)
        jitter_y = np.random.randn(TOTAL_FRAMES) * (TILE_HEIGHT / 4)

        trajectory_x = np.clip(trajectory_x + jitter_x, 0, FRAME_WIDTH - 1)
        trajectory_y = np.clip(trajectory_y + jitter_y, 0, FRAME_HEIGHT - 1)

        # Sample every 5th frame
        sampled_frame_idx = 0
        for full_frame_idx in range(0, TOTAL_FRAMES, FRAME_SAMPLE_RATE):
            # Current center of attention at this frame
            cx = int(trajectory_x[full_frame_idx])
            cy = int(trajectory_y[full_frame_idx])

            # Generate heatmaps for this sampled frame
            fused_saliency = np.zeros((FRAME_HEIGHT, FRAME_WIDTH), dtype=np.float32)

            for heatmap_idx in range(NUM_HEATMAPS):
                # Random sigma for variety in saliency spread
                sigma = FRAME_WIDTH / (6 + np.random.rand() * 4)  # Varies between /6 and /10

                # Add small random offset for each heatmap
                offset_x = np.random.randint(-TILE_WIDTH//2, TILE_WIDTH//2)
                offset_y = np.random.randint(-TILE_HEIGHT//2, TILE_HEIGHT//2)

                center_x = np.clip(cx + offset_x, 0, FRAME_WIDTH - 1)
                center_y = np.clip(cy + offset_y, 0, FRAME_HEIGHT - 1)

                # Create Gaussian saliency map
                y, x = np.ogrid[:FRAME_HEIGHT, :FRAME_WIDTH]
                heatmap = np.exp(-((x - center_x)**2 + (y - center_y)**2) / (2 * sigma**2))

                # Store heatmap
                heatmaps_4d[video_idx, sampled_frame_idx, heatmap_idx] = heatmap

                # Accumulate saliency (equal weight, let model learn optimal fusion)
                fused_saliency += heatmap

            # Find the tile with maximum saliency
            max_y, max_x = np.unravel_index(fused_saliency.argmax(), fused_saliency.shape)
            tile_x = min(max_x // TILE_WIDTH, TILES_X - 1)
            tile_y = min(max_y // TILE_HEIGHT, TILES_Y - 1)
            tile_idx = tile_coords_to_index(tile_x, tile_y)

            # Store tile index for this sampled frame
            tile_indices_1d[video_idx, sampled_frame_idx] = tile_idx

            sampled_frame_idx += 1

    print(f"\nData generation complete!")
    print(f"  Heatmaps 4D shape: {heatmaps_4d.shape}")
    print(f"  Tile indices shape: {tile_indices_1d.shape}")
    print(f"  Memory usage: {heatmaps_4d.nbytes / 1e6:.1f} MB\n")

    return heatmaps_4d, tile_indices_1d


def flatten_video_data(heatmaps_4d, tile_indices_1d):
    """
    Flatten video data from (videos, frames, ...) to (total_frames, ...)

    Args:
        heatmaps_4d: shape (num_videos, frames_per_video, num_heatmaps, H, W)
        tile_indices_1d: shape (num_videos, frames_per_video)

    Returns:
        heatmaps: shape (total_frames, num_heatmaps, H, W)
        tile_indices: shape (total_frames,)
    """
    num_videos, frames_per_video = tile_indices_1d.shape
    total_frames = num_videos * frames_per_video

    # Reshape heatmaps: (videos, frames, heatmaps, H, W) -> (total_frames, heatmaps, H, W)
    heatmaps = heatmaps_4d.reshape(total_frames, NUM_HEATMAPS, FRAME_HEIGHT, FRAME_WIDTH)

    # Flatten tile indices: (videos, frames) -> (total_frames,)
    tile_indices = tile_indices_1d.reshape(total_frames)

    return heatmaps, tile_indices


# Main execution
if __name__ == "__main__":
    print(f"\n{'='*60}")
    print("360° Video Saliency-Based Tile Predictor")
    print(f"{'='*60}\n")

    print(f"Configuration:")
    print(f"  Frame size: {FRAME_WIDTH}x{FRAME_HEIGHT} (25% of 4K)")
    print(f"  Grid: {TILES_X}x{TILES_Y} = {NUM_TILES} tiles")
    print(f"  Tile size: {TILE_WIDTH}x{TILE_HEIGHT}")
    print(f"  Heatmaps per frame: {NUM_HEATMAPS}")
    print(f"  Video: {VIDEO_DURATION}s @ {FPS}fps = {TOTAL_FRAMES} total frames")
    print(f"  Sampling: Every {FRAME_SAMPLE_RATE}th frame = {FRAMES_PER_VIDEO} sampled frames\n")

    # Generate synthetic video data - just 1 video for memory efficiency
    num_videos = 1  # 1 video of 3 seconds
    heatmaps_4d, tile_indices_1d = generate_synthetic_video_data(num_videos=num_videos)

    print(f"{'='*60}")
    print("Data Structure:")
    print(f"{'='*60}")
    print(f"4D Heatmaps Matrix: {heatmaps_4d.shape}")
    print(f"  [videos, sampled_frames, heatmaps, height, width]")
    print(f"  [1, 36, 9, 480, 960]")
    print(f"  - 1 video")
    print(f"  - 36 sampled frames (every 5th frame from 180 total @ 60 FPS)")
    print(f"  - 9 heatmaps per frame (7 audio + 2 video)")
    print(f"  - 480x960 resolution (25% of 4K)")
    print(f"\n1D Tile Index Array: {tile_indices_1d.shape}")
    print(f"  [videos, sampled_frames]")
    print(f"  [1, 36]")
    print(f"  - 1 user's viewing data")
    print(f"  - 36 tile indices (sampled every 5th frame, range 0-143)")
    print(f"  - Each index represents which tile user is looking at")
    print(f"{'='*60}\n")

    # Flatten data for training
    heatmaps_flat, tile_indices_flat = flatten_video_data(heatmaps_4d, tile_indices_1d)

    print(f"Flattened for Training:")
    print(f"  Heatmaps: {heatmaps_flat.shape}")
    print(f"  Tile indices: {tile_indices_flat.shape}\n")

    # Create dataset
    dataset = SaliencyTileDataset(heatmaps_flat, tile_indices_flat)

    # Split
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

    print(f"Dataset splits:")
    print(f"  Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}\n")

    # Initialize model
    model = HeatmapFusionCNN(num_heatmaps=NUM_HEATMAPS, num_tiles=NUM_TILES).to(device)

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model parameters: {total_params:,}\n")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    # Training
    num_epochs = 15
    best_val_acc = 0
    results = []

    print("Training started...\n")
    start_time = time.time()

    for epoch in range(num_epochs):
        epoch_start = time.time()

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, avg_tile_dist = validate(model, val_loader, criterion, device)

        scheduler.step(val_loss)
        epoch_time = time.time() - epoch_start

        results.append([
            epoch + 1,
            f"{train_loss:.4f}",
            f"{train_acc:.1f}%",
            f"{val_loss:.4f}",
            f"{val_acc:.1f}%",
            f"{avg_tile_dist:.2f}",
            f"{epoch_time:.1f}s"
        ])

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        print(f"Epoch {epoch+1}/{num_epochs}: "
              f"Train {train_acc:.1f}% | Val {val_acc:.1f}% | "
              f"Dist {avg_tile_dist:.2f} | {epoch_time:.1f}s")

        # Clear cache periodically
        if epoch % 3 == 0 and device == "cuda":
            torch.cuda.empty_cache()

    total_time = time.time() - start_time

    # Test evaluation
    print("\nEvaluating on test set...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, test_acc, test_tile_dist = validate(model, test_loader, criterion, device)

    print(f"\n{'='*60}")
    print("Training Complete!")
    print(f"{'='*60}\n")

    print(f"Best Val Accuracy: {best_val_acc:.2f}%")
    print(f"Test Accuracy: {test_acc:.2f}%")
    print(f"Test Avg Tile Distance: {test_tile_dist:.2f} tiles")
    print(f"Total Training Time: {total_time:.1f}s ({total_time/60:.1f} min)\n")

    # Training history table
    headers = ['Epoch', 'Train Loss', 'Train Acc', 'Val Loss', 'Val Acc', 'Tile Dist', 'Time']
    print(tabulate(results, headers=headers, tablefmt='simple'))

    # Inference speed test
    print(f"\n{'='*60}")
    print("Inference Speed Test")
    print(f"{'='*60}\n")

    model.eval()
    test_input = torch.randn(1, NUM_HEATMAPS, FRAME_HEIGHT, FRAME_WIDTH).to(device)

    # Warmup
    with torch.no_grad():
        for _ in range(10):
            _ = model(test_input)

    # Measure
    if device == "cuda":
        torch.cuda.synchronize()

    inference_times = []
    with torch.no_grad():
        for _ in range(100):
            start = time.time()
            output = model(test_input)
            if device == "cuda":
                torch.cuda.synchronize()
            inference_times.append(time.time() - start)

    avg_inf_ms = np.mean(inference_times) * 1000
    fps = 1000 / avg_inf_ms

    print(f"Average inference: {avg_inf_ms:.2f}ms")
    print(f"Throughput: {fps:.1f} FPS")
    print(f"Can process: {fps/FPS:.1f}x realtime")

    pred_tile = output.argmax(1).item()
    x, y = tile_index_to_coords(pred_tile)
    print(f"Sample prediction: Tile {pred_tile} at ({x}, {y})")

    print(f"\n{'='*60}")
    print("Model saved as 'best_model.pth'")
    print(f"{'='*60}\n")

    # Sample video analysis - show all sampled frames
    print(f"{'='*60}")
    print("User Viewing Data - Video 0 (3 seconds, every 5th frame)")
    print(f"{'='*60}")
    print(f"Sample# | Frame# | Time(s) | Tile Index | Tile Grid (x,y)")
    print(f"{'-'*60}")
    for i in range(FRAMES_PER_VIDEO):
        tile_idx = tile_indices_1d[0, i]
        tx, ty = tile_index_to_coords(tile_idx)
        actual_frame = i * FRAME_SAMPLE_RATE
        time_sec = actual_frame / FPS
        if i % max(1, FRAMES_PER_VIDEO//12) == 0:  # Show ~12 samples
            print(f"{i:7d} | {actual_frame:6d} | {time_sec:6.2f}  | {tile_idx:10d} | ({tx:2d}, {ty:2d})")
    print(f"{'='*60}\n")

    # Memory cleanup
    del model, train_loader, val_loader, test_loader
    gc.collect()
    if device == "cuda":
        torch.cuda.empty_cache()
        print(f"GPU memory freed: {torch.cuda.memory_allocated()/1e9:.2f} GB in use")

Using device: cpu

360° Video Saliency-Based Tile Predictor

Configuration:
  Frame size: 960x480 (25% of 4K)
  Grid: 16x9 = 144 tiles
  Tile size: 60x53
  Heatmaps per frame: 9
  Video: 3s @ 60fps = 180 total frames
  Sampling: Every 5th frame = 36 sampled frames


Generating Synthetic Video Data

Videos: 1
Duration: 3s @ 60 FPS = 180 total frames
Sampling: Every 5th frame = 36 sampled frames
Heatmap resolution: 960x480 (25% of 4K)
Total sampled frames: 36

Generating video 1/1...

Data generation complete!
  Heatmaps 4D shape: (1, 36, 9, 480, 960)
  Tile indices shape: (1, 36)
  Memory usage: 597.2 MB

Data Structure:
4D Heatmaps Matrix: (1, 36, 9, 480, 960)
  [videos, sampled_frames, heatmaps, height, width]
  [1, 36, 9, 480, 960]
  - 1 video
  - 36 sampled frames (every 5th frame from 180 total @ 60 FPS)
  - 9 heatmaps per frame (7 audio + 2 video)
  - 480x960 resolution (25% of 4K)

1D Tile Index Array: (1, 36)
  [videos, sampled_frames]
  [1, 36]
  - 1 user's viewing data
  - 36 